In [ ]:
### THIS NOTEBOOK HAS THE PYTHON CODE FOR DATA-CLEANING OF THE MHA DATASET ###
### Class BDAT-1011, Group4 -Predictive Policing Project

In [ ]:
# Initial Setup steps:

In [1]:
pip install jupyterlab_execute_time

Note: you may need to restart the kernel to use updated packages.


In [2]:
%timeit
import os

# Set JAVA_HOME to Java 11
os.environ["JAVA_HOME"] = "C:\\Program Files\\Java\\jdk-11"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "\\bin;" + os.environ["PATH"]


In [5]:
from pyspark import SparkContext

sc = SparkContext.getOrCreate()
    

In [6]:
#Import import variables and libraries:
import pyspark
from pyspark import SparkContext
from pyspark.sql import SQLContext

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('MHAData').getOrCreate()
import os

In [7]:
#read in files path to dataset and define schema

file_path = "C:/Users/knick/Desktop/BDAT/1011-MRP/MRP-1003/Data Cleaning/mental-health-apprehensions-refreshed May5_2024.csv"

df_inferred = spark.read.csv(file_path, header=True, inferSchema=True)
df_inferred.printSchema() 


root
 |-- _id: integer (nullable = true)
 |-- EVENT_UNIQUE_ID: string (nullable = true)
 |-- REPORT_DATE: date (nullable = true)
 |-- REPORT_YEAR: integer (nullable = true)
 |-- REPORT_MONTH: string (nullable = true)
 |-- REPORT_DOW: string (nullable = true)
 |-- REPORT_DOY: integer (nullable = true)
 |-- REPORT_DAY: integer (nullable = true)
 |-- REPORT_HOUR: integer (nullable = true)
 |-- OCC_DATE: date (nullable = true)
 |-- OCC_YEAR: integer (nullable = true)
 |-- OCC_MONTH: string (nullable = true)
 |-- OCC_DOY: integer (nullable = true)
 |-- OCC_DAY: integer (nullable = true)
 |-- OCC_DOW: string (nullable = true)
 |-- OCC_HOUR: integer (nullable = true)
 |-- DIVISION: string (nullable = true)
 |-- PREMISES_TYPE: string (nullable = true)
 |-- APPREHENSION_TYPE: string (nullable = true)
 |-- SEX: string (nullable = true)
 |-- AGE_COHORT: string (nullable = true)
 |-- HOOD_158: string (nullable = true)
 |-- NEIGHBOURHOOD_158: string (nullable = true)
 |-- HOOD_140: string (nullable

In [ ]:
# (A) Missing Values :

In [8]:
#Check for missing values in dataset:

import pandas as pd

#Load our dataset into a pandas dataframe:
mha_data = pd.read_csv('C:/Users/knick/Desktop/BDAT/1011-MRP/MRP-1003/Data Cleaning/mental-health-apprehensions-refreshed May5_2024.csv')

# Peak at the first few rows of our df:
print(mha_data.head())

   _id EVENT_UNIQUE_ID REPORT_DATE  REPORT_YEAR REPORT_MONTH REPORT_DOW  \
0    1  GO-20141263946         NaN         2014      January  Wednesday   
1    2  GO-20141261121         NaN         2014      January  Wednesday   
2    3  GO-20141261310         NaN         2014      January  Wednesday   
3    4  GO-20141263993         NaN         2014      January  Wednesday   
4    5  GO-20141262098         NaN         2014      January  Wednesday   

   REPORT_DOY  REPORT_DAY  REPORT_HOUR OCC_DATE  ...  OCC_HOUR DIVISION  \
0           1           1           19      NaN  ...        19      D42   
1           1           1            5      NaN  ...         5      D14   
2           1           1            8      NaN  ...         6      NSA   
3           1           1           19      NaN  ...        19      D12   
4           1           1           11      NaN  ...        11      D11   

   PREMISES_TYPE          APPREHENSION_TYPE     SEX  AGE_COHORT HOOD_158  \
0          House  Mha 

In [8]:
#Next, calculate how many missing fields we have in the df:

# Calculate the total number of rows in the dataset
total_rows = len(mha_data)

#Calculate the Total missing values of each field:
missing_values = mha_data.isnull().sum()
print('Total Missing Values: \n')
print(missing_values)
print()

# Calculate the percentage of missing values for each column
missing_by_percentage = (missing_values / total_rows) * 100
print('Total Missing Values: \n')
print(missing_by_percentage)




Total Missing Values: 

_id                      0
EVENT_UNIQUE_ID          0
REPORT_DATE          91012
REPORT_YEAR              0
REPORT_MONTH             0
REPORT_DOW               0
REPORT_DOY               0
REPORT_DAY               0
REPORT_HOUR              0
OCC_DATE             91002
OCC_YEAR                 0
OCC_MONTH                0
OCC_DOY                  0
OCC_DAY                  0
OCC_DOW                  0
OCC_HOUR                 0
DIVISION                 0
PREMISES_TYPE            0
APPREHENSION_TYPE        0
SEX                      0
AGE_COHORT               0
HOOD_158                 0
NEIGHBOURHOOD_158        0
HOOD_140                 0
NEIGHBOURHOOD_140        0
dtype: int64

Total Missing Values: 

_id                   0.000000
EVENT_UNIQUE_ID       0.000000
REPORT_DATE          83.567015
REPORT_YEAR           0.000000
REPORT_MONTH          0.000000
REPORT_DOW            0.000000
REPORT_DOY            0.000000
REPORT_DAY            0.000000
REPORT_HOUR    

In [18]:
# Do the actual cleaning here - address missing and invalid data in Report_Date:

import pandas as pd
import numpy as np

# copy our original dataset:
rawData = mha_data

# Define a function that will validate Report_Date and/or combine report_year, report_month, report_day into a date :
def Build_Report_Date(row):
    # check if the current value of Report_Date is valid and within our date range of 2014-2024 :
    try:
        if pd.isna(row['REPORT_DATE']):
            try:
                # Construct date from year, month, day
                return pd.Timestamp(year=row['REPORT_YEAR'], 
                                month=pd.to_datetime(row['REPORT_MONTH'], format='%B').month, 
                                day=row['REPORT_DAY'])
            except ValueError:
                # Return NaN if there's an error in date creation (e.g., invalid date)
                return np.nan
        
        # If Report_Date is valid and within a reasonable timeframe, return it
        current_date = pd.to_datetime(row['REPORT_DATE'])
        if current_date.year >= 2014 and current_date.year <= 2024:
            return current_date

    except (ValueError, TypeError):
        # If there's an error in trying to build the date, return NaN
        return np.nan


In [19]:
# Now apply the function results to each row of dataset     
rawData['REPORT_DATE'] = rawData.apply(Build_Report_Date, axis=1)

# Check what rows are still NaN (missing or invalid) :
print(rawData[rawData['REPORT_DATE'].isna()])

# We will save our cleaned data to a new CSV
cleaned_dataset = "C:/Users/knick/Desktop/BDAT/1011-MRP/MRP-1003/Data Cleaning/cleaned_Report_Date.csv"
rawData.to_csv(cleaned_dataset, index=False)

Empty DataFrame
Columns: [_id, EVENT_UNIQUE_ID, REPORT_DATE, REPORT_YEAR, REPORT_MONTH, REPORT_DOW, REPORT_DOY, REPORT_DAY, REPORT_HOUR, OCC_DATE, OCC_YEAR, OCC_MONTH, OCC_DOY, OCC_DAY, OCC_DOW, OCC_HOUR, DIVISION, PREMISES_TYPE, APPREHENSION_TYPE, SEX, AGE_COHORT, HOOD_158, NEIGHBOURHOOD_158, HOOD_140, NEIGHBOURHOOD_140]
Index: []

[0 rows x 25 columns]


In [20]:
#Below we verify if there are any remaining NULL values in Report_Date :

print(rawData[['REPORT_YEAR', 'REPORT_MONTH', 'REPORT_DAY']].isna().sum())
print(rawData[['REPORT_DATE']].isna().sum())

REPORT_YEAR     0
REPORT_MONTH    0
REPORT_DAY      0
dtype: int64
REPORT_DATE    0
dtype: int64


In [21]:
# NOW WE WILL DO THE SAME FOR OCC_DATE -> For missing values, we will build a date using 
#OCC_YEAR, OCCMONTH and OCC_DAY

# copy our original dataset:
rawData2 = mha_data

# Function to validate and possibly correct the OCC_DATE
def Build_Occ_Date(row):

    # check if the current value of Report_Date is valid and within our date range of 2014-2024 :
    try:
        if pd.isna(row['OCC_DATE']):
            try:
                # Construct date from year, month, day
                return pd.Timestamp(year=row['OCC_YEAR'], 
                                month=pd.to_datetime(row['OCC_MONTH'], format='%B').month, 
                                day=row['OCC_DAY'])
            except ValueError:
                # Return NaN if there's an error in date creation (e.g., invalid date)
                return np.nan

        # If Report_Date is valid and within a reasonable timeframe, return it
        current_date = pd.to_datetime(row['OCC_DATE'])
        if current_date.year >= 2014 and current_date.year <= 2024:
            return current_date

    except (ValueError, TypeError):
        # If there's an error in trying to build the date, return NaN
        return np.nan
            

# Apply the function to each row
rawData2['OCC_DATE'] = rawData2.apply(Build_Occ_Date, axis=1)

# Display rows where OCC_DATE was corrected
print(rawData2[rawData2['OCC_DATE'].isna()])

# Save the cleaned data back to CSV
cleaned_dataset2 = "C:/Users/knick/Desktop/BDAT/1011-MRP/MRP-1003/Data Cleaning/cleaned_Update_Occ_Date.csv"
rawData2.to_csv(cleaned_dataset2, index=False)





         _id EVENT_UNIQUE_ID REPORT_DATE  REPORT_YEAR REPORT_MONTH REPORT_DOW  \
234      235  GO-20141335282  2014-01-13         2014      January     Monday   
314      315  GO-20141351010  2014-01-16         2014      January   Thursday   
908      909  GO-20141531563  2014-02-14         2014     February     Friday   
10900  10901   GO-2015962360  2015-06-09         2015         June    Tuesday   

       REPORT_DOY  REPORT_DAY  REPORT_HOUR OCC_DATE  ...  OCC_HOUR DIVISION  \
234            13          13           13      NaT  ...        12      D14   
314            16          16            2      NaT  ...        22      D55   
908            45          14           21      NaT  ...        21      D43   
10900         160           9            1      NaT  ...        21      D12   

       PREMISES_TYPE          APPREHENSION_TYPE     SEX  AGE_COHORT HOOD_158  \
234        Apartment  Mha Sec 17 (Power Of App)    Male         65+       84   
314        Apartment  Mha Sec 17 (Powe

In [23]:
print(rawData2[['OCC_YEAR', 'OCC_MONTH', 'OCC_DAY']].isna().sum())
print(rawData2[['OCC_DATE']].isna().sum())

OCC_YEAR     0
OCC_MONTH    0
OCC_DAY      0
dtype: int64
OCC_DATE    4
dtype: int64
